# I. $LU$ factorization of a square matrix

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [3]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [4]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)

6

In [5]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [6]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  1.110e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -2.819e-16  0.000e+00  0.000e+00  8.080e-04  1.902e-03]
 [ 0.000e+00  3.369e-16  0.000e+00 -1.541e-18  2.168e-19 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  2.220e-16 -1.110e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00  2.220e-16 -5.551e-17 -1.665e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00 -1.110e-16  2.776e-16 -2.776e-16  5.551e-17]
 

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [7]:
a1 = a.copy()
a1[1, 1] = 3

Resulting matix still has full rank, but the naive LU routine breaks down.

In [8]:
np.linalg.matrix_rank(a1)

6

In [9]:
l, u = diy_lu(a1)

print(l, u)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


C:\Users\vadim\Anaconda_3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
C:\Users\vadim\Anaconda_3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in matmul
C:\Users\vadim\Anaconda_3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in matmul
C:\Users\vadim\Anaconda_3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [10]:
def lead_minors(a):
    for i in range(2, a.shape[0]):
        minor = a[:(i-1), :(i-1)]
        if np.linalg.det(minor) == 0:
            ok = False
            break
        else:
            ok = True
    return ok
lead_minors(a), lead_minors(a1)
#Таким образом, один из главных миноров матрицы а1 обнуляется, поэтому данное разложение для нее не работает

(True, False)

### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

In [11]:
from numpy.linalg import inv
def diy_new(a):
    N = a.shape[0] 
    u = a.copy()
    PL = np.eye(N)
    P = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        pivot = np.argmax(np.abs(u[j:, j]))
        p = np.zeros((N, N))
        for m in range(N):
            for n in range(N):
                if m != pivot+j and m != j:
                    if m == n:
                        p[m, n] = 1
                p[j, pivot+j] = 1
                p[pivot+j, j] = 1
        u = p @ u
        P = p @ P
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u
        lam[j+1:, j] = gamma
        PL = PL @ inv(p) @ lam
    L = P @ PL
    return L, u, P

In [12]:
def check(L, u, P):
    return inv(P) @ L @ u

Проверим, правильно ли работает программа

In [16]:
from numpy import allclose
L1, u1, P1 = diy_new(a1)
L, u, P = diy_new(a)
my_ans = allclose(check(L1, u1, P1), a1), allclose(check(L, u, P), a)
print(my_ans)

(True, True)
